Code written by Margot Wagner
Last updated 09/11/21

In [19]:
import requests
import ipywidgets as widgets
from ipywidgets import interact, interactive
from IPython.display import display
import pandas as pd

Every patient has an existing database containing their current prescriptions and prescription information.

In [59]:
def drug_info(generic_name, reason_for_prescription, prescriber, FOIs):
    """
    Given a certain drug, this function creates a series with the requested fields of interest (FOI)

    :param generic_name: drug's generic name
    :param reason_for_prescription: reason for prescription of drug 
    :param prescriber: prescriber name/info

    :return Series: series containing FOIs for drug of interest
    """
    
    # fields of interest
    fields = FOIs 

    # access prescription in openFDA
    URL = 'https://api.fda.gov/drug/label.json?api_key=M2JwidRocyUdc8gvmAKOpXKa7eIoX43tSWaN2Hip&search=openfda.generic_name.exact:{}&limit=1'.format(generic_name.upper())
    data = requests.get(URL).json()

    # initialize fields of interest (FOI) for given prescription
    drug_info = [generic_name.upper()]

    for result in data.get('results', []):
        # FOIs
        for f in fields[1:-2]:

            # if openFDA has FOI, populate it, otherwise input ND (no data)
            # normal category
            if 'openfda.' in f:
                # subsearch req for pharm class
                subsearch = f.split('.')[1]

                if subsearch in result['openfda'].keys():
                    search = result['openfda'][subsearch]
                    drug_info.append(search)
                else:
                    drug_info.append('ND')
            elif f in result.keys():
                search = result[f]
                drug_info.append(search)
            else:
                drug_info.append('ND')
        
        # add prescription information
        drug_info.append(reason_for_prescription)
        drug_info.append(prescriber)

        # convert to series
        series = pd.Series(drug_info, index = fields)

    return series
    

Populate user's drug database with existing prescriptions.

In [60]:
prescriptions = ['DICLOFENAC', 'GABAPENTIN', 'DULOXETINE']
reasons = ['pain relief', 'nerve pain', 'antidepressant']
physician = ['chronic pain physician', 'primary care physician', 'psychiatrist']
fields = ['generic_name', 'description', 'openfda.pharm_class_epc', 'how_supplied', 'drug_interactions', 'indications_and_usage', 'reason_for_prescription', 'prescriber']  # fields of interest                                                                    

# initialize drug info dataframe
drug_df = pd.DataFrame(columns = fields)

# populate dataframe
for i in range(len(prescriptions)):
    drug_series = drug_info(prescriptions[i], reasons[i], physician[i], fields)
    drug_df = drug_df.append(drug_series, ignore_index=True)


In [61]:
drug_df


,generic_name,description,openfda.pharm_class_epc,how_supplied,drug_interactions,indications_and_usage,reason_for_prescription,prescriber
0,DICLOFENAC,ND,ND,ND,ND,[Use(s)  for the temporary relief of arthriti...,pain relief,chronic pain physician
1,GABAPENTIN,"[DESCRIPTION SECTION Gabapentin tablets, USP a...",[Anti-epileptic Agent [EPC]],"[HOW SUPPLIED SECTION Gabapentin Tablets, USP ...",ND,[INDICATIONS & USAGE SECTION Postherpetic Neur...,nerve pain,primary care physician
2,DULOXETINE,"[11 DESCRIPTION Duloxetine hydrochloride, USP ...",ND,[16 HOW SUPPLIED/STORAGE AND HANDLING Product:...,[7 DRUG INTERACTIONS Both CYP1A2 and CYP2D6 ar...,[1 INDICATIONS & USAGE Duloxetine delayed-rele...,antidepressant,psychiatrist


In [62]:
# Add clonazepam (can be something else)
style = {'description_width': 'initial'} # fit description length

new_drug = widgets.Text(
    value = 'clonazepam',
    placeholder='Enter generic name',
    description='New prescription:',
    disabled=False,
    style=style
)
display(new_drug)

new_reason = widgets.Text(
    value = 'panic attacks',
    placeholder='Enter reason',
    description='Reason for prescription:',
    disabled=False,
    style=style
)
display(new_reason)

new_physician = widgets.Text(
    value = 'psychiatrist',
    placeholder='Enter name',
    description='Prescribing physician:',
    disabled=False,
    style=style
)
display(new_physician)

Text(value='clonazepam', description='New prescription:', placeholder='Enter generic name', style=DescriptionS…

Text(value='panic attacks', description='Reason for prescription:', placeholder='Enter reason', style=Descript…

Text(value='psychiatrist', description='Prescribing physician:', placeholder='Enter name', style=DescriptionSt…

In [77]:
print("Attempting to add {} (prescribed by {} for {})".format(new_drug.value, new_physician.value, new_reason.value))

Attempting to add clonazepam (prescribed by psychiatrist for insomnia)


In [78]:
new_series = drug_info(new_drug.value, new_reason.value, new_physician.value, fields)
new_series

generic_name                                                      CLONAZEPAM
description                [DESCRIPTION Clonazepam, a benzodiazepine, is ...
openfda.pharm_class_epc                               [Benzodiazepine [EPC]]
how_supplied               [HOW SUPPLIED Clonazepam tablets USP 0.5 mg ar...
drug_interactions          [Drug Interactions: Effect of Concomitant Use ...
indications_and_usage      [INDICATIONS AND USAGE Seizure Disorders: Clon...
reason_for_prescription                                             insomnia
prescriber                                                      psychiatrist
dtype: object

In [79]:
# Check for drug interactions
interactions = new_series['drug_interactions'][0].upper()

# search for existing drugs in drug interactions
for p in prescriptions:
    if p in interactions:
        print("WARNING: {} MAY INTERACT WITH {}. PLEASE REVIEW".format(p, new_drug.value))


In [83]:
# Check for deprescriptions for all drugs
drug_class = new_series['openfda.pharm_class_epc'][0].upper()
reason = new_series['reason_for_prescription'].upper()

# check pharm_class_epc to see if it matches big 5
# if so, follow that algorithm (https://deprescribing.org/resources/deprescribing-guidelines-algorithms/)
if 'BENZODIAZEPINE' in drug_class:
    print('RUNNING BENZODIAZEPINE DEPRESCRIPTION ALGORITHM')
    # benzo deprescription algo
    if 'INSOMNIA' in reason:
        print('DEPRESCRIBING MAY BE RECOMMENDED. CLICK TO REVIEW.')
    else:
        print('DEPRESCRIBING NOT RECOMMENDED')
elif 'PROTON PUMP INHIBITOR' in drug_class:
    print('RUNNING PROTON PUMP INHIBITOR DEPRESCRIPTION ALGORITHM')
elif 'ANTIHYPERGLYCEMIC' in drug_class:
    print('RUNNING ANTIHYPERGLYCEMIC DEPRESCRIPTION ALGORITHM')
elif 'ANTIPSYCHOTIC' in drug_class:
    print('RUNNING ANTIPSYCHOTIC DEPRESCRIPTION ALGORITHM')
elif 'CHOLINESTERASE INHIBITORS' in drug_class:
    print('RUNNING CHOLINESTERASE INHIBITORS DEPRESCRIPTION ALGORITHM')

RUNNING BENZODIAZEPINE DEPRESCRIPTION ALGORITHM
DEPRESCRIBING MAY BE RECOMMENDED. CLICK TO REVIEW.
